In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

seed = 2023
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.experimental.numpy.random.seed(seed)
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
os.environ["TF_DETERMINISTIC_OPS"] = "1"
# Set a fixed value for the hash seed
os.environ["PYTHONHASHSEED"] = str(seed)
import dolfin as df
import time
from utils import *
from utils_training import *
import prepare_data
from utils_compare_methods import *
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
import vedo
import vedo.dolfin as vdf
from prepare_data import rotate, outside_ball
import pandas as pd

sns.set_theme()
sns.set_context("paper")
colors = sns.color_palette("mako").as_hex()
my_cmap = sns.color_palette("viridis", as_cmap=True)

%load_ext autoreload
%autoreload 2


In [ ]:
save_figs = True
small_data = False

data = DataLoader(small_data)
agent_H2 = Agent(data)
agent_L2 = Agent(data, loss_level=0)
agent_H1 = Agent(data, loss_level=1)
if not (os.path.exists(f"./images/")) and save_figs:
    os.makedirs(f"./images/")


file = open("./models_H1/best_model/best_epoch.txt")
for y in file.read().split(" "):
    if y.isdigit():
        best_epoch = int(y)

best_epoch_H1 = best_epoch

file = open("./models/best_model/best_epoch.txt")
for y in file.read().split(" "):
    if y.isdigit():
        best_epoch = int(y)

best_epoch_H2 = best_epoch

file = open("./models_L2/best_model/best_epoch.txt")
for y in file.read().split(" "):
    if y.isdigit():
        best_epoch = int(y)

best_epoch_L2 = best_epoch

print(f"{best_epoch_H2=}    {best_epoch_L2=}    {best_epoch_H1=}")

In [ ]:
agent_H2.model.load_weights(f"./models/model_{best_epoch_H2}/model_weights")
agent_L2.model.load_weights(f"./models_L2/model_{best_epoch_L2}/model_weights")
agent_H1.model.load_weights(f"./models_H1/model_{best_epoch_H1}/model_weights")


def compare_std_phi_fem_and_fno(
    param, agent_H2, agent_L2, agent_H1, Plot=False
):
    standard_solver = StandardFEMSolver(params=param)
    solver = PhiFemSolver_error(nb_cell=64 - 1, params=param)
    mu0, mu1, sigma, x_0, y_0, lx, ly, theta, alpha, beta = param[0]

    u_ref, V_ref, dx_ref = standard_solver.solve_one(0, None, True)
    (
        u_phi_fem,
        V_phi_fem,
        dx_phi_fem,
        _h_phi,
        cell_selection_phi,
        submesh_construction_phi,
        ghost_cell_selection_phi,
        resolution_time_phi,
    ) = solver.solve_one(0)

    (
        u_std,
        _h_std,
        construction_time_standard,
        resolution_time_standard,
    ) = standard_solver.solve_one(0, 64, False)

    u_phi_fem_proj = df.project(
        u_phi_fem,
        V_ref,
        solver_type="gmres",
        preconditioner_type="hypre_amg",
    )

    l2_error_phi_fem = (
        df.assemble((((u_ref - u_phi_fem_proj)) ** 2) * dx_ref) ** (0.5)
    ) / (df.assemble((((u_ref)) ** 2) * dx_ref) ** (0.5))

    u_std_fem_proj = df.project(
        u_std, V_ref, solver_type="gmres", preconditioner_type="hypre_amg"
    )
    l2_error_std_fem = (
        df.assemble((((u_ref - u_std_fem_proj)) ** 2) * dx_ref) ** (0.5)
    ) / (df.assemble((((u_ref)) ** 2) * dx_ref) ** (0.5))

    F = generate_F_numpy(mu0, mu1, sigma, 64) / data.max_norm_F
    G = generate_G_numpy(alpha, beta, 64)
    phi = generate_phi_numpy(x_0, y_0, lx, ly, theta, 64)
    X = generate_manual_new_data_numpy(phi, F, G)
    Y_H2 = agent_H2.model.call(X)
    predicted_solution_H2 = X[:, :, :, 1] * Y_H2[:, :, :, 0] + X[:, :, :, 2]
    predicted_solution_H2 = np.reshape(predicted_solution_H2, (64, 64))
    predicted_sol_H2_fenics = convert_numpy_matrix_to_fenics(
        predicted_solution_H2, 64, 1
    )

    predicted_sol_H2_fenics_proj_V_ref = df.project(
        predicted_sol_H2_fenics,
        V_ref,
        solver_type="gmres",
        preconditioner_type="hypre_amg",
    )

    l2_error_fno_H2 = (
        df.assemble(
            (((u_ref - predicted_sol_H2_fenics_proj_V_ref)) ** 2) * dx_ref
        )
        ** (0.5)
    ) / (df.assemble((((u_ref)) ** 2) * dx_ref) ** (0.5))

    Y_L2 = agent_L2.model.call(X)
    predicted_solution_L2 = X[:, :, :, 1] * Y_L2[:, :, :, 0] + X[:, :, :, 2]
    predicted_solution_L2 = np.reshape(predicted_solution_L2, (64, 64))
    predicted_sol_L2_fenics = convert_numpy_matrix_to_fenics(
        predicted_solution_L2, 64, 1
    )
    predicted_sol_L2_fenics_proj_V_ref = df.project(
        predicted_sol_L2_fenics,
        V_ref,
        solver_type="gmres",
        preconditioner_type="hypre_amg",
    )

    l2_error_fno_L2 = (
        df.assemble(
            (((u_ref - predicted_sol_L2_fenics_proj_V_ref)) ** 2) * dx_ref
        )
        ** (0.5)
    ) / (df.assemble((((u_ref)) ** 2) * dx_ref) ** (0.5))

    Y_H1 = agent_H1.model.call(X)
    predicted_solution_H1 = X[:, :, :, 1] * Y_H1[:, :, :, 0] + X[:, :, :, 2]
    predicted_solution_H1 = np.reshape(predicted_solution_H1, (64, 64))
    predicted_sol_H1_fenics = convert_numpy_matrix_to_fenics(
        predicted_solution_H1, 64, 1
    )
    predicted_sol_H1_fenics_proj_V_ref = df.project(
        predicted_sol_H1_fenics,
        V_ref,
        solver_type="gmres",
        preconditioner_type="hypre_amg",
    )

    l2_error_fno_H1 = (
        df.assemble(
            (((u_ref - predicted_sol_H1_fenics_proj_V_ref)) ** 2) * dx_ref
        )
        ** (0.5)
    ) / (df.assemble((((u_ref)) ** 2) * dx_ref) ** (0.5))

    print(f"{l2_error_fno_H2=}")
    print(f"{l2_error_fno_L2=}")
    print(f"{l2_error_fno_H1=}")
    print(f"{l2_error_phi_fem=}")
    print(f"{l2_error_std_fem=}")

    return (
        l2_error_phi_fem,
        l2_error_std_fem,
        l2_error_fno_H2,
        l2_error_fno_L2,
        l2_error_fno_H1,
    )

In [ ]:
if not (os.path.exists("./errors_compare_fnos")):
    os.makedirs("./errors_compare_fnos")

In [ ]:
save_list_error = True
(
    L2_error_fno_H2,
    L2_error_fno_L2,
    L2_error_fno_H1,
    L2_error_phi_fem,
    L2_error_std_fem,
) = (
    [],
    [],
    [],
    [],
    [],
)

F, phi, G, params = new_create_FG_numpy(300, 64)
epoch = best_epoch
indices = list(range(0, len(params)))
for index in indices:
    print(f"Iter : {index+1}/{len(params)}")
    (
        l2_error_phi_fem,
        l2_error_std_fem,
        l2_error_fno_H2,
        l2_error_fno_L2,
        l2_error_fno_H1,
    ) = compare_std_phi_fem_and_fno(
        np.array([params[index]]),
        agent_H2=agent_H2,
        agent_L2=agent_L2,
        agent_H1=agent_H1,
    )

    L2_error_phi_fem.append(l2_error_phi_fem)
    L2_error_fno_H2.append(l2_error_fno_H2)
    L2_error_std_fem.append(l2_error_std_fem)
    L2_error_fno_L2.append(l2_error_fno_L2)
    L2_error_fno_H1.append(l2_error_fno_H1)
    if save_list_error:
        np.save(
            "./errors_compare_fnos/L2_error_phi_fem.npy",
            np.array([L2_error_phi_fem]),
        )
        np.save(
            "./errors_compare_fnos/L2_error_std_fem.npy",
            np.array([L2_error_std_fem]),
        )
        np.save(
            "./errors_compare_fnos/L2_error_fno_H2.npy",
            np.array([L2_error_fno_H2]),
        )
        np.save(
            "./errors_compare_fnos/L2_error_fno_L2.npy",
            np.array([L2_error_fno_L2]),
        )
        np.save(
            "./errors_compare_fnos/L2_error_fno_H1.npy",
            np.array([L2_error_fno_H1]),
        )

In [ ]:
L2_error_phi_fem = np.load("./errors_compare_fnos/L2_error_phi_fem.npy")
L2_error_std_fem = np.load("./errors_compare_fnos/L2_error_std_fem.npy")
L2_error_fno_H2 = np.load("./errors_compare_fnos/L2_error_fno_H2.npy")
L2_error_fno_L2 = np.load("./errors_compare_fnos/L2_error_fno_L2.npy")
L2_error_fno_H1 = np.load("./errors_compare_fnos/L2_error_fno_H1.npy")

In [ ]:
error_tab = []
error_tab.append(L2_error_phi_fem)
error_tab.append(L2_error_std_fem)
error_tab.append(L2_error_fno_L2)
error_tab.append(L2_error_fno_H1)
error_tab.append(L2_error_fno_H2)

abs_str = [
    "$\phi$-FEM",
    "Standard FEM",
    r"FNO $\mathcal{L}_0$ loss",
    r"FNO $\mathcal{L}_{H_1}$ loss",
    r"FNO $\mathcal{L}$ loss",
]
errors = np.array(error_tab[:])
errors = np.reshape(errors, (5, np.shape(errors)[-1]))
print(np.shape(errors))
dataframe = pd.DataFrame(errors.transpose(), columns=abs_str)

sns.set(font_scale=1.1)

plt.figure(figsize=(10, 6))
sns.boxplot(data=dataframe, palette="ch:s=.0,rot=0.0,dark=0.5")
plt.yscale("log")
plt.xlabel("Method", fontsize=16)
plt.ylabel("Relative $L^2$ error", fontsize=16)
plt.grid(axis="y", visible=True, which="both")

plt.tight_layout()
if save_figs:
    plt.savefig(f"./images/boxplots_new_data_compare_method_FNOs.png")
plt.show()
